# Forecasting the Breeding Bird Survey

In [1]:
library(forecast)
library(dplyr)
library(MODISTools)

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: timeDate
This is forecast 6.1 


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



## Initial data setup

Load the BBS data and limit it to sites with contiguous time-series from 2000 to 2014 (the range spanned by the MODIS data).

In [2]:
data <- read.csv("./data/bbs_data.csv")
colnames(data)[3] <- "long"
contig_modern_routes <- read.csv('./data/contig_modern_routes.csv')
colnames(contig_modern_routes) <- c('site_id')
data_modern <- semi_join(data, contig_modern_routes)
data_modern <- filter(data_modern, year >= 2000)

#data <- data_modern
head(data)

Joining by: "site_id"


,site_id,lat,long,year,species_id,abundance
1,2001,34.86869,-87.60414,1967,2010,2
2,2001,34.86869,-87.60414,1967,2020,1
3,2001,34.86869,-87.60414,1967,2730,8
4,2001,34.86869,-87.60414,1967,3870,19
5,2001,34.86869,-87.60414,1967,4440,6
6,2001,34.86869,-87.60414,1967,5970,4


## Extract the long contiguous time-series

In [3]:
filtered_ts <- function(df){
    full_ts <- seq(from=min(df$year), to=max(df$year), by=1)
    contig_pos <- na.contiguous(match(full_ts,unique(df$year)))
    filter(df, year %in% full_ts[contig_pos])
    }

min_ts_length = 20

data_by_site <- group_by(data, site_id)
contig_ts <- do(data_by_site, filtered_ts(.))
contig_ts_by_site <- group_by(contig_ts, site_id)
contig_ts_length <- summarize(contig_ts_by_site, n_years = n_distinct(year))
long_ts <- filter(contig_ts_length, n_years >= min_ts_length)
contig_ts_long <- semi_join(contig_ts, long_ts)
head(contig_ts_long)

Joining by: "site_id"


,site_id,lat,long,year,species_id,abundance
1,2007,34.86804,-86.20304,1974,2010,1
2,2007,34.86804,-86.20304,1974,2730,4
3,2007,34.86804,-86.20304,1974,2890,24
4,2007,34.86804,-86.20304,1974,3131,2
5,2007,34.86804,-86.20304,1974,3160,48
6,2007,34.86804,-86.20304,1974,3250,10


In [4]:
data_by_site_yr <- group_by(contig_ts_long, site_id, year)
richness <- summarise(data_by_site_yr, richness = n_distinct(species_id))
head(richness)

,site_id,year,richness
1,2007,1974,52
2,2007,1978,55
3,2007,1979,59
4,2007,1980,54
5,2007,1981,56
6,2007,1982,44


In [5]:
richness_by_site <- group_by(richness, site_id)
cat("There are", n_groups(richness_by_site), "continuous time-series with at least" , min_ts_length, "years of data")

There are 900 continuous time-series with at least 20 years of data

## NDVI time-series data

In [7]:
data_modern <- filter(data, year >= 2000)
data_by_site_modern <- group_by(data_modern, site_id)
contig_ts_modern <- do(data_by_site_modern, filtered_ts(.))
contig_ts_by_site_modern <- group_by(contig_ts_modern, site_id)
contig_ts_length_modern <- summarize(contig_ts_by_site_modern, n_years = n_distinct(year))
long_ts_modern <- filter(contig_ts_length_modern, n_years >= 15)
contig_ts_long_modern <- semi_join(contig_ts_modern, long_ts_modern)

data_by_site_yr_ndvi <- group_by(contig_ts_long_modern, site_id, lat, long, year)
richness_ndvi <- summarise(data_by_site_yr_ndvi, richness = n_distinct(species_id))
colnames(richness_ndvi)[4] <- 'start.date'
richness_ndvi$end.date <- richness_ndvi$start.date
head(richness_ndvi)

Joining by: "site_id"


,site_id,lat,long,start.date,richness,end.date
1,2001,34.86869,-87.60414,2000,61,2000
2,2001,34.86869,-87.60414,2001,58,2001
3,2001,34.86869,-87.60414,2002,61,2002
4,2001,34.86869,-87.60414,2003,66,2003
5,2001,34.86869,-87.60414,2004,62,2004
6,2001,34.86869,-87.60414,2005,59,2005


## Forecasting

Generate forecasts for naive, average, and ARIMA time-series models for each site.

In [18]:
lag <- 5
out <- do(richness_by_site,
          cast_naive = naive(.$richness[1:(length(.$richness) - lag - 1)], lag)$mean,
          cast_avg = meanf(.$richness[1:(length(.$richness) - lag - 1)], lag)$mean,
          cast_arima = forecast(auto.arima(.$richness[1:(length(.$richness) - lag - 1)], seasonal = FALSE), h = lag)$mean,
          test_set = (.$richness[(length(.$richness) - lag):length(.$richness)])
         )
head(out)

,site_id,cast_naive,cast_avg,cast_arima,test_set
1,2007,"41, 41, 41, 41, 41","48.83333, 48.83333, 48.83333, 48.83333, 48.83333","42.83177, 41.71331, 42.39623, 41.97925, 42.23385","46, 49, 43, 48, 46, 45"
2,2013,"68, 68, 68, 68, 68","62.71429, 62.71429, 62.71429, 62.71429, 62.71429","68, 68, 68, 68, 68","72, 66, 74, 78, 69, 75"
3,2014,"62, 62, 62, 62, 62","61.38235, 61.38235, 61.38235, 61.38235, 61.38235","63.37128, 59.44316, 63.33138, 59.48265, 63.29229","67, 67, 70, 64, 69, 69"
4,2015,"65, 65, 65, 65, 65","55.54839, 55.54839, 55.54839, 55.54839, 55.54839","65, 65, 65, 65, 65","62, 63, 70, 56, 69, 64"
5,2017,"57, 57, 57, 57, 57","60.23077, 60.23077, 60.23077, 60.23077, 60.23077","58.13802, 59.04460, 57.96523, 58.09546, 58.57508","60, 65, 64, 60, 54, 51"
6,2019,"52, 52, 52, 52, 52","61.43333, 61.43333, 61.43333, 61.43333, 61.43333","55.07103, 55.21501, 56.35169, 56.84630, 57.43939","54, 54, 56, 55, 52, 57"


## Forecasting experiments

In [149]:
examp <- filter(richness, site_id == 2007)
forecast <- naive(examp$richness)
examp$richness[1:length(examp$richness)]

[1] 52 55 59 54 56 44 51 43 48 45 46 49 46 49 47 50 44 41 46 49 43 48 46 45

In [153]:
lag <- 5
out <- do(richness_by_site,
          cast_naive = naive(.$richness[1:(length(.$richness) - lag - 1)], lag)$mean,
          cast_avg = meanf(.$richness[1:(length(.$richness) - lag - 1)], lag)$mean,
          cast_arima = forecast(auto.arima(.$richness[1:(length(.$richness) - lag - 1)], seasonal = FALSE), h = lag)$mean
         )
head(out)

,site_id,cast_naive,cast_avg,cast_arima
1,2007,"41, 41, 41, 41, 41","48.83333, 48.83333, 48.83333, 48.83333, 48.83333","ARIMA(1,1,0) , -0.6105897, 15.23058, 0.03283248, TRUE, -47.50331, 99.00662, 1, 0, 0, 0, 1, 1, 0, 0.05199996, 2.375882, 5.831769, -2.557641, -1.052948, -10.77882, -0.3270759, -3.725872, 0.1152827, 0.05294829, -0.831769, 3.61059, -1.168231, 1.168231, -0.168231, 1.778821, -4.168231, -6.663538, auto.arima(x = structure(list(x = structure(c(52L, 55L, 59L, , 54L, 56L, 44L, 51L, 43L, 48L, 45L, 46L, 49L, 46L, 49L, 47L, 50L, , 44L, 41L), .Tsp = c(1, 18, 1), class = ""ts"")), .Names = ""x"", row.names = c(NA, , -18L), class = ""data.frame""), seasonal = FALSE), .$richness[1:(length(.$richness) - lag - 1)], 0, 0, 17, -0.6105897, 1, 1, 1, -3, 44, 0, 1.237628e-22, 1.237628e-22, -1.237628e-22, -0.6105897, 1, 0, 1, 1, 0, 0, 0, 0, 1, 7.556831e-23, 7.556831e-23, -1.237628e-22, 100.673, 99.86376, 52, 55, 59, 54, 56, 44, 51, 43, 48, 45, 46, 49, 46, 49, 47, 50, 44, 41, 80, 95, 42.83177, 41.71331, 42.39623, 41.97925, 42.23385, 37.83034, 36.34605, 35.81287, 34.87365, 34.37008, 35.18274, 33.50479, 32.32784, 31.11218, 30.20726, 47.8332, 47.08057, 48.97959, 49.08484, 50.09762, 50.4808, 49.92183, 52.46462, 52.84631, 54.26045, 52, 55, 59, 54, 56, 44, 51, 43, 48, 45, 46, 49, 46, 49, 47, 50, 44, 41, structure(c(52L, 55L, 59L, 54L, 56L, 44L, 51L, 43L, 48L, 45L, , 46L, 49L, 46L, 49L, 47L, 50L, 44L, 41L), .Tsp = c(1, 18, 1), class = ""ts""), 51.948, 52.62412, 53.16823, 56.55764, 57.05295, 54.77882, 51.32708, 46.72587, 47.88472, 44.94705, 46.83177, 45.38941, 47.16823, 47.83177, 47.16823, 48.22118, 48.16823, 47.66354, 0.05199996, 2.375882, 5.831769, -2.557641, -1.052948, -10.77882, -0.3270759, -3.725872, 0.1152827, 0.05294829, -0.831769, 3.61059, -1.168231, 1.168231, -0.168231, 1.778821, -4.168231, -6.663538"
2,2013,"68, 68, 68, 68, 68","62.71429, 62.71429, 62.71429, 62.71429, 62.71429","ARIMA(0,1,0) , 25.38462, -39.46813, 80.93626, 0, 0, 0, 0, 1, 1, 0, 0.05399997, 2, 0, 3, -8, 6, 7, 9, -7, 4, -3, 0, 3, -2, auto.arima(x = structure(list(x = structure(c(54L, 56L, 56L, , 59L, 51L, 57L, 64L, 73L, 66L, 70L, 67L, 67L, 70L, 68L), .Tsp = c(1, , 14, 1), class = ""ts"")), .Names = ""x"", row.names = c(NA, -14L), class = ""data.frame""), , seasonal = FALSE), .$richness[1:(length(.$richness) - lag - 1)], 0, 0, 13, 1, 1, 1, -2, 70, 0, 7.420876e-23, 7.420876e-23, -7.420876e-23, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, -7.420876e-23, 81.50121, 81.2999, 54, 56, 56, 59, 51, 57, 64, 73, 66, 70, 67, 67, 70, 68, 80, 95, 68, 68, 68, 68, 68, 61.54314, 58.86862, 56.81639, 55.08628, 53.56202, 58.12508, 54.03476, 50.89614, 48.25017, 45.91902, 74.45686, 77.13138, 79.18361, 80.91372, 82.43798, 77.87492, 81.96524, 85.10386, 87.74983, 90.08098, 54, 56, 56, 59, 51, 57, 64, 73, 66, 70, 67, 67, 70, 68, structure(c(54L, 56L, 56L, 59L, 51L, 57L, 64L, 73L, 66L, 70L, , 67L, 67L, 70L, 68L), .Tsp = c(1, 14, 1), class = ""ts""), 53.946, 54, 56, 56, 59, 51, 57, 64, 73, 66, 70, 67, 67, 70, 0.05399997, 2, 0, 3, -8, 6, 7, 9, -7, 4, -3, 0, 3, -2"
3,2014,"62, 62, 62, 62, 62","61.38235, 61.38235, 61.38235, 61.38235, 61.38235","ARIMA(1,0,1) with non-zero mean, -0.9898434, 0.9089038, 61.39719, 7.718563, 0.0005863913, -0.001877887, 0.0002329058, -0.001877887, 0.008725607, -0.001222975, 0.0002329058, -0.001222975, 0.2093455, TRUE, TRUE, TRUE, -83.51122, 175.0224, 1, 1, 0, 0, 1, 0, 0, 0.5238519, -0.1954261, -2.462802, -1.594173, -1.349081, -3.497748, 4.235736, 2.351658, 0.0879606, 0.1192999, 4.064929, 3.472008, 0.04663461, -1.833505, -0.1113667, -3.687377, -0.4082971, -1.427205, 0.5232645, -5.277328, 3.056018, 1.383292, -3.04373, -7.011159, -2.339757, -1.664176, 1.751275, 3.576741, 0.9392286, -0.6741108, 5.841451, 0.8199285, 1.474496, 2.828721, auto.arima(x = structure(list(x = structure(c(62L, 61L, 59L, , 60L, 60L, 58L, 66L, 63L, 62L, 61L, 66L, 64L, 62L, 59L, 62L, 57L, , 62L, 59L, 63L, 55L, 66L, 61L, 60L, 53L, 61L, 58L, 65L, 63L, 64L, , 59L, 69L, 60L, 65L, 62L), .Tsp = c(1, 34, 1), class = ""ts"

In [14]:
out$cast[[1]]

Time Series:
Start = 11 
End = 15 
Frequency = 1 
[1] 58 58 58 58 58

In [150]:
train_set <- 1:(length(examp$richness)-lag - 1)
test_set <- (length(examp$richness)-lag):length(examp$richness)
model <- auto.arima(examp$richness[train_set], seasonal = FALSE)
cast_arima <- forecast(model, h = 5)
cast_naive <- naive(examp$richness[train_set], h = 5)
cast_avg <- meanf(examp$richness[train_set], h = 5)
examp$richness[test_set] - cast_arima$mean
examp$richness[test_set] - cast_naive$mean
examp$richness[test_set] - cast_avg$mean
model

Time Series:
Start = 20 
End = 24 
Frequency = 1 
[1]  6.156072 -1.836086  4.421392  1.627654  1.128673

Time Series:
Start = 20 
End = 24 
Frequency = 1 
[1]  3 -3  2  0 -1

[1]  0.3157895 -5.6842105 -0.6842105 -2.6842105 -3.6842105

Series: examp$richness[train_set] 
ARIMA(1,1,0)                    

Coefficients:
          ar1
      -0.6312
s.e.   0.1767

sigma^2 estimated as 14.9:  log likelihood=-50.11
AIC=104.21   AICc=105.01   BIC=105.99

In [8]:
## NDVI

richness_ndvi_oneyear <- filter(richness_ndvi, start.date == 2005)

MODISSummaries(LoadDat = richness_ndvi_oneyear, Dir = "./data/modisdata", Product = "MOD13Q1", Bands = c("250m_16_days_NDVI"),
               ValidRange = c(-2000,10000),
               NoDataFill = -3000, ScaleFactor = 0.0001,
               StartDate = TRUE, QualityScreen = FALSE,
               Max = TRUE, Mean = TRUE)

MODIS data files from ./data/modisdata will be summarised.
Summary files will be written to the same directory.
Processing file 1 of 681...
Processing file 2 of 681...
Processing file 3 of 681...
Processing file 4 of 681...
Processing file 5 of 681...
Processing file 6 of 681...
Processing file 7 of 681...
Processing file 8 of 681...
Processing file 9 of 681...
Processing file 10 of 681...
Processing file 11 of 681...
Processing file 12 of 681...
Processing file 13 of 681...
Processing file 14 of 681...
Processing file 15 of 681...
Processing file 16 of 681...
Processing file 17 of 681...
Processing file 18 of 681...
Processing file 19 of 681...
Processing file 20 of 681...
Processing file 21 of 681...
Processing file 22 of 681...
Processing file 23 of 681...
Processing file 24 of 681...
Processing file 25 of 681...
Processing file 26 of 681...
Processing file 27 of 681...
Processing file 28 of 681...
Processing file 29 of 681...
Processing file 30 of 681...
Processing file 31 of 681..

Warning message:
In min(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to min; returning InfWarning message:
In max(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to max; returning -InfWarning message:
In min(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to min; returning InfWarning message:
In max(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to max; returning -Inf

Processing file 319 of 681...
Processing file 320 of 681...
Processing file 321 of 681...
Processing file 322 of 681...
Processing file 323 of 681...
Processing file 324 of 681...
Processing file 325 of 681...
Processing file 326 of 681...
Processing file 327 of 681...
Processing file 328 of 681...
Processing file 329 of 681...
Processing file 330 of 681...
Processing file 331 of 681...
Processing file 332 of 681...
Processing file 333 of 681...
Processing file 334 of 681...
Processing file 335 of 681...
Processing file 336 of 681...
Processing file 337 of 681...
Processing file 338 of 681...
Processing file 339 of 681...
Processing file 340 of 681...
Processing file 341 of 681...
Processing file 342 of 681...
Processing file 343 of 681...
Processing file 344 of 681...
Processing file 345 of 681...
Processing file 346 of 681...
Processing file 347 of 681...
Processing file 348 of 681...
Processing file 349 of 681...
Processing file 350 of 681...
Processing file 351 of 681...
Processing

Warning message:
In min(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to min; returning InfWarning message:
In max(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to max; returning -Inf

Processing file 439 of 681...
Processing file 440 of 681...
Processing file 441 of 681...
Processing file 442 of 681...
Processing file 443 of 681...
Processing file 444 of 681...
Processing file 445 of 681...
Processing file 446 of 681...
Processing file 447 of 681...
Processing file 448 of 681...
Processing file 449 of 681...
Processing file 450 of 681...
Processing file 451 of 681...
Processing file 452 of 681...
Processing file 453 of 681...
Processing file 454 of 681...
Processing file 455 of 681...
Processing file 456 of 681...
Processing file 457 of 681...
Processing file 458 of 681...
Processing file 459 of 681...
Processing file 460 of 681...
Processing file 461 of 681...
Processing file 462 of 681...
Processing file 463 of 681...
Processing file 464 of 681...
Processing file 465 of 681...
Processing file 466 of 681...
Processing file 467 of 681...
Processing file 468 of 681...
Processing file 469 of 681...
Processing file 470 of 681...
Processing file 471 of 681...
Processing

Warning message:
In min(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to min; returning InfWarning message:
In max(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to max; returning -InfWarning message:
In min(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to min; returning InfWarning message:
In max(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to max; returning -Inf

Processing file 617 of 681...
Processing file 618 of 681...
Processing file 619 of 681...
Processing file 620 of 681...
Processing file 621 of 681...
Processing file 622 of 681...
Processing file 623 of 681...
Processing file 624 of 681...
Processing file 625 of 681...
Processing file 626 of 681...
Processing file 627 of 681...
Processing file 628 of 681...
Processing file 629 of 681...
Processing file 630 of 681...
Processing file 631 of 681...
Processing file 632 of 681...
Processing file 633 of 681...
Processing file 634 of 681...
Processing file 635 of 681...
Processing file 636 of 681...
Processing file 637 of 681...
Processing file 638 of 681...
Processing file 639 of 681...
Processing file 640 of 681...
Processing file 641 of 681...
Processing file 642 of 681...
Processing file 643 of 681...
Processing file 644 of 681...
Processing file 645 of 681...
Processing file 646 of 681...
Processing file 647 of 681...
Processing file 648 of 681...
Processing file 649 of 681...
Processing

Warning message:
In min(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to min; returning InfWarning message:
In max(as.numeric(band.time.series[, i]) * ScaleFactor, na.rm = TRUE): no non-missing arguments to max; returning -Inf

Processing file 665 of 681...
Processing file 666 of 681...
Processing file 667 of 681...
Processing file 668 of 681...
Processing file 669 of 681...
Processing file 670 of 681...
Processing file 671 of 681...
Processing file 672 of 681...
Processing file 673 of 681...
Processing file 674 of 681...
Processing file 675 of 681...
Processing file 676 of 681...
Processing file 677 of 681...
Processing file 678 of 681...
Processing file 679 of 681...
Processing file 680 of 681...
Processing file 681 of 681...
Done! Check the 'MODIS Summary' and 'MODIS Data' output files.


In [13]:
ts <- MODISTimeSeries(Dir = "./data/modisdata/", Band = "250m_16_days_NDVI", Simplify = TRUE)

Simplify == TRUE, but not all tiles have the same number of rows so cannot be
simplified into one matrix. Returning data as an array instead.


In [22]:
ts[[500]].

,Lat43.2370751Lon-83.2549351Samp9Line9_pixel1,Lat43.2370751Lon-83.2549351Samp9Line9_pixel2,Lat43.2370751Lon-83.2549351Samp9Line9_pixel3,Lat43.2370751Lon-83.2549351Samp9Line9_pixel4,Lat43.2370751Lon-83.2549351Samp9Line9_pixel5,Lat43.2370751Lon-83.2549351Samp9Line9_pixel6,Lat43.2370751Lon-83.2549351Samp9Line9_pixel7,Lat43.2370751Lon-83.2549351Samp9Line9_pixel8,Lat43.2370751Lon-83.2549351Samp9Line9_pixel9,Lat43.2370751Lon-83.2549351Samp9Line9_pixel10,Lat43.2370751Lon-83.2549351Samp9Line9_pixel11,Lat43.2370751Lon-83.2549351Samp9Line9_pixel12,Lat43.2370751Lon-83.2549351Samp9Line9_pixel13,Lat43.2370751Lon-83.2549351Samp9Line9_pixel14,Lat43.2370751Lon-83.2549351Samp9Line9_pixel15,Lat43.2370751Lon-83.2549351Samp9Line9_pixel16,Lat43.2370751Lon-83.2549351Samp9Line9_pixel17,Lat43.2370751Lon-83.2549351Samp9Line9_pixel18,Lat43.2370751Lon-83.2549351Samp9Line9_pixel19,Lat43.2370751Lon-83.2549351Samp9Line9_pixel20,Lat43.2370751Lon-83.2549351Samp9Line9_pixel21,Lat43.2370751Lon-83.2549351Samp9Line9_pixel22,Lat43.2370751Lon-83.2549351Samp9Line9_pixel23,Lat43.2370751Lon-83.2549351Samp9Line9_pixel24,Lat43.2370751Lon-83.2549351Samp9Line9_pixel25,Lat43.2370751Lon-83.2549351Samp9Line9_pixel26,Lat43.2370751Lon-83.2549351Samp9Line9_pixel27,Lat43.2370751Lon-83.2549351Samp9Line9_pixel28,Lat43.2370751Lon-83.2549351Samp9Line9_pixel29,Lat43.2370751Lon-83.2549351Samp9Line9_pixel30,Lat43.2370751Lon-83.2549351Samp9Line9_pixel31,Lat43.2370751Lon-83.2549351Samp9Line9_pixel32,Lat43.2370751Lon-83.2549351Samp9Line9_pixel33,Lat43.2370751Lon-83.2549351Samp9Line9_pixel34,Lat43.2370751Lon-83.2549351Samp9Line9_pixel35,Lat43.2370751Lon-83.2549351Samp9Line9_pixel36,Lat43.2370751Lon-83.2549351Samp9Line9_pixel37,Lat43.2370751Lon-83.2549351Samp9Line9_pixel38,Lat43.2370751Lon-83.2549351Samp9Line9_pixel39,Lat43.2370751Lon-83.2549351Samp9Line9_pixel40,Lat43.2370751Lon-83.2549351Samp9Line9_pixel41,Lat43.2370751Lon-83.2549351Samp9Line9_pixel42,Lat43.2370751Lon-83.2549351Samp9Line9_pixel43,Lat43.2370751Lon-83.2549351Samp9Line9_pixel44,Lat43.2370751Lon-83.2549351Samp9Line9_pixel45,Lat43.2370751Lon-83.2549351Samp9Line9_pixel46,Lat43.2370751Lon-83.2549351Samp9Line9_pixel47,Lat43.2370751Lon-83.2549351Samp9Line9_pixel48,Lat43.2370751Lon-83.2549351Samp9Line9_pixel49,Lat43.2370751Lon-83.2549351Samp9Line9_pixel50,Lat43.2370751Lon-83.2549351Samp9Line9_pixel51,Lat43.2370751Lon-83.2549351Samp9Line9_pixel52,Lat43.2370751Lon-83.2549351Samp9Line9_pixel53,Lat43.2370751Lon-83.2549351Samp9Line9_pixel54,Lat43.2370751Lon-83.2549351Samp9Line9_pixel55,Lat43.2370751Lon-83.2549351Samp9Line9_pixel56,Lat43.2370751Lon-83.2549351Samp9Line9_pixel57,Lat43.2370751Lon-83.2549351Samp9Line9_pixel58,Lat43.2370751Lon-83.2549351Samp9Line9_pixel59,Lat43.2370751Lon-83.2549351Samp9Line9_pixel60,Lat43.2370751Lon-83.2549351Samp9Line9_pixel61,Lat43.2370751Lon-83.2549351Samp9Line9_pixel62,Lat43.2370751Lon-83.2549351Samp9Line9_pixel63,Lat43.2370751Lon-83.2549351Samp9Line9_pixel64,Lat43.2370751Lon-83.2549351Samp9Line9_pixel65,Lat43.2370751Lon-83.2549351Samp9Line9_pixel66,Lat43.2370751Lon-83.2549351Samp9Line9_pixel67,Lat43.2370751Lon-83.2549351Samp9Line9_pixel68,Lat43.2370751Lon-83.2549351Samp9Line9_pixel69,Lat43.2370751Lon-83.2549351Samp9Line9_pixel70,Lat43.2370751Lon-83.2549351Samp9Line9_pixel71,Lat43.2370751Lon-83.2549351Samp9Line9_pixel72,Lat43.2370751Lon-83.2549351Samp9Line9_pixel73,Lat43.2370751Lon-83.2549351Samp9Line9_pixel74,Lat43.2370751Lon-83.2549351Samp9Line9_pixel75,Lat43.2370751Lon-83.2549351Samp9Line9_pixel76,Lat43.2370751Lon-83.2549351Samp9Line9_pixel77,Lat43.2370751Lon-83.2549351Samp9Line9_pixel78,Lat43.2370751Lon-83.2549351Samp9Line9_pixel79,Lat43.2370751Lon-83.2549351Samp9Line9_pixel80,Lat43.2370751Lon-83.2549351Samp9Line9_pixel81
A2000049,4242,4242,4289,3367,3367,3421,3421,3275,3275,3032,3032,2825,2825,3040,3040,2780,3275,3286,3263,3180,2823,3040,3040,2780,3309,3309,3509,3180,3180,3214,3650,3650,3880,3880,3792,3792,3091,3191,3583,3583,3644,3644,3792,3792,3389,3098,3453,3583,3644,3644,3847,3847,3365,3154,345

In [12]:
time.series <- data.frame(lat = c(51.41363, 51.41421),
                          long = c(-0.64875, -0.641607),
                          start.date = c(2002, 2002),
                          end.date = c(2004, 2004),
                          ID = c(1, 2))
head(time.series)

,lat,long,start.date,end.date,ID
1,51.41363,-0.64875,2002,2004,1
2,51.41421,-0.641607,2002,2004,2
